###Clean and transform the Data, write to unity catalog SILVER table

In [0]:
from pyspark.sql.functions import col, hour
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import IntegerType

schema: str = "so_schema"

# Change Date and Date_et_heure_de_restitution to timestamp format
df_silver = (spark.read
             .table(f"dbx_training.{schema}.bronze_objets_trouves")
             .withColumn("Date", to_timestamp("Date", "yyyy-MM-dd'T'HH:mm:ssXXX"))
             .withColumn("Date_et_heure_de_restitution", to_timestamp("Date_et_heure_de_restitution", "yyyy-MM-dd'T'HH:mm:ssXXX")))

# Filter non-numeric values in Code_UIC
df_silver = df_silver.filter(col("Code_UIC").rlike("^\d+(\.\d+)?$"))

# Change Code_UIC type to int
df_silver = df_silver.withColumn("Code_UIC", col("Code_UIC").cast(IntegerType()))

# Drop rows with missing values
df_silver = df_silver.dropna()

# Deduplicate data
df_silver = df_silver.dropDuplicates()

# Add a new column Hour
df_silver = df_silver.withColumn('Hour', hour(col('Date')))

# Create and store the data in a delta table "silver_objets_trouves"
df_silver.write.mode("overwrite").saveAsTable(f"dbx_training.{schema}.silver_objets_trouves")